In [1]:
import pandas as pd

In [ ]:
df = pd.read_csv("sarcasm-annos-emnlp13.tsv", sep="\t", names=["tweet_id", "sarcastic"])
df

In [3]:
from TwitterAPI import TwitterAPI
consumer_key = ''
consumer_secret = ''
access_token_key = ''
access_token_secret = ''
api = TwitterAPI(consumer_key, consumer_secret, access_token_key, access_token_secret)

In [8]:
api

In [ ]:
r = api.request('statuses/show/:{}'.format(210462857140252672))
print(r.text)

In [13]:
page = 0 #change page variable from 0 to 3 to get all 3000 tweets
with open('scraped_tweets_'+str(page)+'.tsv', 'w') as the_file:
    for row_index in range(page*750, (page+1)*750):
        r = df.iloc[row_index]
        tweet_id = r["tweet_id"]
        sarcastic = r["sarcastic"] == "SARCASM"
        r = api.request('statuses/show/:{}'.format(tweet_id))
        tweet_text = r.text
        the_file.write(str(tweet_text) + '\t' + str(int(sarcastic)) + '\n')

In [ ]:
df_api_0 = df = pd.read_csv("scraped_tweets_0.tsv", sep="\t", names=["tweet", "sarcastic"])
df_api_1 = df = pd.read_csv("scraped_tweets_0.tsv", sep="\t", names=["tweet", "sarcastic"])
df_api_2 = df = pd.read_csv("scraped_tweets_0.tsv", sep="\t", names=["tweet", "sarcastic"])
df_api_3 = df = pd.read_csv("scraped_tweets_0.tsv", sep="\t", names=["tweet", "sarcastic"])
df_api = pd.concat([df_api_0, df_api_1, df_api_2, df_api_3])
df_api

In [4]:
import json
import re

def tweet_text(row):
    parsed_json = json.loads(row["tweet"])
    return parsed_json["text"]

insensitive_sarcasm = re.compile(re.escape('#sarcasm'), re.IGNORECASE)
insensitive_sarcastic = re.compile(re.escape('#sarcastic'), re.IGNORECASE)

def clean_tweet_text(row):
    return insensitive_sarcasm.sub('', insensitive_sarcastic.sub('', row["tweet_text"]))
    

In [ ]:

df_clean = df_api[df_api["tweet"].str.startswith('{"errors":') == False]
#https://stackoverflow.com/questions/26886653/pandas-create-new-column-based-on-values-from-other-columns
df_clean['tweet_text'] = df_clean.apply (lambda row: tweet_text(row),axis=1)
df_clean['clean_tweet_text'] = df_clean.apply (lambda row: clean_tweet_text(row),axis=1)
df_clean

In [6]:
df_clean.to_csv("clean_tweet_data.tsv", sep='\t')